# Python “黑箱”：输入与输出

## 输入输出基础
最简单直接的输入来自键盘操作，比如下面这个例子:  

In [29]:
# input() 函数 输入的类型永远都是字符串比型（str）#
name = input('your name:')
gender = input('you are a boy?(y/n)')
### 输入 ###
#print(name, gender)
welcome_str = 'Welcome to the matrix {prefix} {name}.'
welcome_dic = {
    'prefix': 'Mr.' if gender == 'y' else 'Mrs',
    'name': name
}
print('authorizing...')
print(welcome_str.format(**welcome_dic))

authorizing...
Welcome to the matrix Mrs .


print() 函数则接受字符串、数字、字典、列表甚至一些自定义类的输出。

In [30]:
a = input()
b = input()

print('a + b = {}'.format(a + b))

print('type of a is{},type of b is{}'.format(type(a),type(b)))

print('a + b = {}'.format(int(a) + int(b)))

a + b = 
type of a is<class 'str'>,type of b is<class 'str'>


ValueError: invalid literal for int() with base 10: ''

把 str 强制转换为 int 请用 int()，转为浮点数请用 float()。而在生产环境中使用强制转换时，请记得加上 try except（即错误和异常处理）。  
Python 对 int 类型没有最大限制（相比之下， C++ 的 int 最大为 2147483647，超过这个数字会产生溢出），但是对 float 类型依然有精度限制。这些特点，除了在一些算法竞赛中要注意，在生产环境中也要时刻提防，避免因为对边界条件判断不清而造成 bug 甚至 0day（危重安全漏洞）。

## 文件输入输出
命令行的输入输出，只是 Python 交互的最基本方式，适用一些简单小程序的交互。而生产级别的 Python 代码，大部分 I/O 则来自于文件、网络、其他进程的消息等等。

 NLP 任务的基本步骤，也就是下面的四步：

读取文件；

+ 去除所有标点符号和换行符，并把所有大写变成小写；

+ 合并相同的词，统计每个词出现的频率，并按照词频从大到小排序；
 
+ 将结果按行输出到文件 out.txt。


In [ ]:
import re
 
# 你不用太关心这个函数
def parse(text):
    # 使用正则表达式去除标点符号和换行符
    text = re.sub(r'[^\w ]', ' ', text)
 
    # 转为小写
    text = text.lower()
    
    # 生成所有单词的列表
    word_list = text.split(' ')
    
    # 去除空白单词
    word_list = filter(None, word_list)
    
    # 生成单词和词频的字典
    word_cnt = {}
    for word in word_list:
        if word not in word_cnt:
            word_cnt[word] = 0
        word_cnt[word] += 1
    
    # 按照词频排序
    sorted_word_cnt = sorted(word_cnt.items(), key=lambda kv: kv[1], reverse=True)
    
    return sorted_word_cnt
 
with open('in.txt', 'r') as fin:
    text = fin.read()
 
word_and_freq = parse(text)
 
with open('out.txt', 'w') as fout:
    for word, freq in word_and_freq:
        fout.write('{} {}\n'.format(word, freq))

## JSON 序列化与实战
SON（JavaScript Object Notation）是一种轻量级的数据交换格式，它的设计意图是把所有事情都用设计的字符串来表示，这样既方便在互联网上传递信息，也方便人进行阅读（相比一些 binary 的协议）。  
设想一个情景，你要向交易所购买一定数额的股票。那么，你需要提交股票代码、方向（买入 / 卖出）、订单类型（市价 / 限价）、价格（如果是限价单）、数量等一系列参数，而这些数据里，有字符串，有整数，有浮点数，甚至还有布尔型变量，全部混在一起并不方便交易所解包。  
JSON ，正能解决这个场景。你可以把它简单地理解为两种黑箱：
* 第一种，输入这些杂七杂八的信息，比如 Python 字典，输出一个字符串；
* 第二种，输入这个字符串，可以输出包含原始信息的 Python 字典。

In [31]:
import json
 
params = {
    'symbol': '123456',
    'type': 'limit',
    'price': 123.4,
    'amount': 23
}
 
params_str = json.dumps(params)
 
print('after json serialization')
print('type of params_str = {}, params_str = {}'.format(type(params_str), params))
 
original_params = json.loads(params_str)
 
print('after json deserialization')
print('type of original_params = {}, original_params = {}'.format(type(original_params), original_params))

after json serialization
type of params_str = <class 'str'>, params_str = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}
after json deserialization
type of original_params = <class 'dict'>, original_params = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}


其中，

* json.dumps() 这个函数，接受 Python 的基本数据类型，然后将其序列化为 string；

* 而 json.loads() 这个函数，接受一个合法字符串，然后将其反序列化为 Python 的基本数据类型。

使用场景：开发一个第三方应用程序时，你可以通过 JSON 将用户的个人配置输出到文件，方便下次程序启动时自动读取。


In [32]:
import json
 
params = {
    'symbol': '123456',
    'type': 'limit',
    'price': 123.4,
    'amount': 23
}
 
with open('params.json', 'w') as fout:
    params_str = json.dump(params, fout)
 
with open('params.json', 'r') as fin:
    original_params = json.load(fin)
 
print('after json deserialization')
print('type of original_params = {}, original_params = {}'.format(type(original_params), original_params))

after json deserialization
type of original_params = <class 'dict'>, original_params = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}


## 总结
主要学习了 Python 的普通 I/O 和文件 I/O，同时了解了 JSON 序列化的基本知识，并通过具体的例子进一步掌握。再次强调一下需要注意的几点：  
* I/O 操作需谨慎，一定要进行充分的错误处理，并细心编码，防止出现编码漏洞；

* 编码时，对内存占用和磁盘占用要有充分的估计，这样在出错时可以更容易找到原因；

* JSON 序列化是很方便的工具，要结合实战多多练习；

* 代码尽量简洁、清晰，哪怕是初学阶段，也要有一颗当元帅的心